In [6]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sk
import re
data_frame = pd.read_csv("../datasets/train.csv")
data_frame.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [7]:
data_frame = data_frame.drop(columns=["id", "severe_toxic", "obscene", "threat", "insult", "identity_hate"])
data_frame

,comment_text,toxic
0,Explanation\r\nWhy the edits made under my use...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\r\nMore\r\nI can't make any real suggestions...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \r\n\r\nThat...,0
159568,"Spitzer \r\n\r\nUmm, theres no actual article ...",0
159569,And it looks like it was actually you who put ...,0


In [24]:
def normalize_single_comment(comment) -> str:
    if comment is None:
        return None
    comment = comment.lower()
    comment = re.sub("'", "", comment)
    comment = " ".join(re.findall("[a-z]+", comment))

    if len(comment.strip(" ")) == 0 or len(comment.rstrip()) == 0:
        return None
    else:
        return comment

In [25]:
data_frame["comment_text"] = data_frame["comment_text"].apply(normalize_single_comment)

data_frame

,comment_text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more i cant make any real suggestions on impro...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159566,and for the second time of asking when your vi...,0
159567,you should be ashamed of yourself that is a ho...,0
159568,spitzer umm theres no actual article for prost...,0
159569,and it looks like it was actually you who put ...,0
